In [31]:
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import HuggingFaceDatasetLoader, YoutubeLoader
from langchain_community.chat_models import ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
from openai import OpenAI
import tiktoken
import numpy as np
import os

In [32]:
load_dotenv()

OPEN_ROUTER_API_KEY = os.getenv("OPEN_ROUTER_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [33]:
# Free Llama 3.1 API via OpenRouter
# Use this instead of OpenAI if you don't have an OpenAI account with credits

openrouter_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPEN_ROUTER_API_KEY
)

In [34]:
pc = Pinecone(api_key=PINECONE_API_KEY)

hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/bert-large-nli-max-tokens")
vectorstore = PineconeVectorStore(index_name="customer-support", embedding=hf_embeddings)
retreiver = vectorstore.as_retriever()

/home/user/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [35]:
template = """Answer the question based only on the following context:
{context}
Question: {question}"""

prompt = ChatPromptTemplate.from_template(template)

In [36]:
chain = (
  RunnableParallel({"context": retreiver, "question": RunnablePassthrough()})
  | prompt
  | openrouter_client.chat.completions.create
  | StrOutputParser()
)

In [44]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GEMINI_API_KEY)
llm.invoke("Hello, world!")

'Greetings, Earthlings!'